In [6]:
import os
import sys
import pathlib
import subprocess
import glob
import pandas as pd
import numpy as np
import xarray as xr

# define filepaths
root = pathlib.Path("..").resolve()
chrtout_path = os.path.join(root,"test","input","geo","NWM_2.1_Sample_Datasets","Pocono_TEST1","example_CHRTOUT")


In [50]:
def drop_all_coords(ds):
    return ds.reset_coords(drop=True)

stack = xr.open_mfdataset(
        os.path.join(chrtout_path,"*"),
        combine="by_coords",
        preprocess=drop_all_coords,
    )

import dask.array as da
x = da.from_array(d, chunks=(1000, 1000))

qTroute = np.random.randn(len(stack.time),len(stack.feature_id))
time = stack.time.values
feature_id = stack.feature_id.values

new_data = xr.DataArray(
    data=qTroute,
    dims=["time","feature_id"],
    coords=dict(
        time = time,
        feature_id=feature_id
    ),
    attrs=dict(
        description="t-route streamflow",
        units="cubic meters per second",
    ),
)


stack["qTroute"] = new_data

In [51]:
stack

<xarray.Dataset>
Dimensions:         (feature_id: 374, reference_time: 1, time: 24)
Coordinates:
  * feature_id      (feature_id) int32 4186117 4186199 ... 4185689 4185589
  * reference_time  (reference_time) datetime64[ns] 2018-01-01T01:00:00
  * time            (time) datetime64[ns] 2018-01-01T02:00:00 ... 2018-01-02T01:00:00
Data variables:
    crs             (time) |S1 b'' b'' b'' b'' b'' b'' ... b'' b'' b'' b'' b''
    order           (time, feature_id) int32 dask.array<chunksize=(1, 374), meta=np.ndarray>
    elevation       (time, feature_id) float32 dask.array<chunksize=(1, 374), meta=np.ndarray>
    streamflow      (time, feature_id) float32 dask.array<chunksize=(1, 374), meta=np.ndarray>
    q_lateral       (time, feature_id) float32 dask.array<chunksize=(1, 374), meta=np.ndarray>
    velocity        (time, feature_id) float32 dask.array<chunksize=(1, 374), meta=np.ndarray>
    qSfcLatRunoff   (time, feature_id) float32 dask.array<chunksize=(1, 374), meta=np.ndarray>
    qBucket         (time, feature_id) float32 dask.array<chunksize=(1, 374), meta=np.ndarray>
    qBtmVertRunoff  (time, feature_id) float32 dask.array<chunksize=(1, 374), meta=np.ndarray>
    qTroute         (time, feature_id) float64 1.651 -0.0908 ... -0.6773 0.3983
Attributes:
    featureType:                timeSeries
    proj4:                      +proj=lcc +units=m +a=6370000.0 +b=6370000.0 ...
    model_initialization_time:  2018-01-01_01:00:00
    station_dimension:          feature_id
    model_output_valid_time:    2018-01-01_02:00:00
    model_total_valid_times:    24
    stream_order_output:        1
    cdm_datatype:               Station
    Conventions:                CF-1.6
    code_version:               
    model_output_type:          channel_rt
    model_configuration:        retrospective
    dev_OVRTSWCRT:              1
    dev_NOAH_TIMESTEP:          3600
    dev_channel_only:           0
    dev_channelBucket_only:     0
    dev:                        dev_ prefix indicates development/internal me...